# Re-train and re-deploy AutoAI pipelines with `ibm-watsonx-ai`

This notebook contains the steps and code to demonstrate support of AI Lifecycle features of the AutoAI model in Watson Machine Learning Service in Watson Machine Learning service. It contains steps and code to work with [ibm-watsonx-ai](https://pypi.python.org/pypi/ibm-watsonx-ai) library available in PyPI repository. It also introduces commands for  training, persisting and deploying model, scoring it, updating the model and redeploying it.

Some familiarity with Python is helpful. This notebook uses Python 3.11.


## Learning goals

The learning goals of this notebook are:

-  List all deprecated and unsupported deployments.
-  Identify AutoAI models that need to be retrained.
-  Work with Watson Machine Learning experiments to re-train AutoAI models.
-  Persist an updated AutoAI model in Watson Machine Learning repository.
-  Redeploy model in-place.
-  Score sample records using client library.


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.	[Deployments state check](#state)
3.  [Identification of model requiring retraining](#identify)
4.	[Experiment re-run](#run)
5.	[Persist trained AutoAI model](#persistence)
6.  [Redeploy and score new version of the model](#redeploy)
7.  [Clean up](#cleanup)
8.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watson-machine-learning" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).
-  Create a <a href="https://console.bluemix.net/catalog/infrastructure/cloud-object-storage" target="_blank" rel="noopener no referrer">Cloud Object Storage (COS)</a> instance (a lite plan is offered and information about how to order storage can be found <a href="https://console.bluemix.net/docs/services/cloud-object-storage/basics/order-storage.html#order-storage" target="_blank" rel="noopener no referrer">here</a>). <br/>**Note: When using Watson Studio, you already have a COS instance associated with the project you are running the notebook in.**


### Install and import the `ibm-watsonx-ai` and dependecies
**Note:** `ibm-watsonx-ai` documentation can be found <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install -U ibm-watsonx-ai | tail -n 1

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud. You need to provide Cloud `API key` and `location`.

**Tip**: Your `Cloud API key` can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below. You can also get a service specific url by going to the [**Endpoint URLs** section of the Watson Machine Learning docs](https://cloud.ibm.com/apidocs/machine-learning).  You can check your instance location in your  <a href="https://cloud.ibm.com/catalog/services/watson-machine-learning" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance details.


You can use [IBM Cloud CLI](https://cloud.ibm.com/docs/cli/index.html) to retrieve the instance `location`.

```
ibmcloud login --apikey API_KEY -a https://cloud.ibm.com
ibmcloud resource service-instance WML_INSTANCE_NAME
```


**NOTE:** You can also get a service specific apikey by going to the [**Service IDs** section of the Cloud Console](https://cloud.ibm.com/iam/serviceids).  From that page, click **Create**, and then copy the created key and paste it in the following cell.


**Action**: Enter your `api_key` and `location` in the following cell.

In [43]:
api_key = 'PUT_YOUR_KEY_HERE'
location = 'us-south'

In [1]:
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    api_key=api_key,
    url='https://' + location + '.ml.cloud.ibm.com'
)

In [2]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials)

### Working with spaces

You need to create a space that will be used for your work. If you do not have a space, you can use [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=wx) to create one.

- Click **New Deployment Space**
- Create an empty space
- Select Cloud Object Storage
- Select Watson Machine Learning instance and press **Create**
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: assign space ID below

You can use the `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

Extract all spaces `id`'s.

In [3]:
spaces_ids = [space['metadata']['id'] for space in client.spaces.get_details()['resources']]

<a id="state"></a>
## 2. Deployments state check
Iterate over spaces and search for `deprecated` and `unsupported` deployments. Next, identify models requiring re-training.

In [ ]:
from ibm_watsonx_ai.lifecycle import SpecStates


for space_id in spaces_ids:
    client.set.default_space(space_id)
    print('****** SPACE', space_id, '******')
    print(client.deployments.get_details(spec_state=SpecStates.DEPRECATED))
    print(client.deployments.get_details(spec_state=SpecStates.UNSUPPORTED))

You can also list deployments under particular space. The output contains `SPEC_STATE` and `SPEC_REPLACEMENT`. Set the working space.

In [49]:
deployment_space_id = 'PROVIDE SPACE_ID HERE'
client.set.default_space(deployment_space_id)

'SUCCESS'

List deployments under this space.

In [ ]:
client.deployments.list()

<a id="identify"></a>
## 3. Identification of model requiring retraining
Pick up deployment of the AutoAI model you wish to retrain. 

**Hint**: You can also do that programatically in the loop sequence over spaces check (`Check the state of your deployments` cell).

**Hint**: You can also use software_specification information (model details) to identify models and deployments that are not yet deprecated but can be retrained (updated software specification is available).

In [51]:
deployment_id = 'PROVIDE DEPLOYMENT_ID HERE'
deployed_model_id = client.deployments.get_details(deployment_id)['entity']['asset']['id']

Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.


#### Extract the deployed model's details (including the pipeline information).

In [52]:
import json

deployed_model_details = client.repository.get_model_details(deployed_model_id)
deployed_pipeline_id = deployed_model_details['entity']['pipeline']['id']
deployed_pipeline_details = client.repository.get_details(deployed_pipeline_id)
experiment_params = deployed_pipeline_details['entity']['document']['pipelines'][0]['nodes'][0]['parameters']
optimization_params = experiment_params['optimization']

print('Experiment parameters:', json.dumps(experiment_params, indent=3))
print('Optimization parameters:', json.dumps(optimization_params, indent=3))

Experiment parameters: {
   "output_logs": true,
   "input_file_separator": ",",
   "stage_flag": true,
   "optimization": {
      "compute_pipeline_notebooks_flag": true,
      "cv_num_folds": 3.0,
      "daub_adaptive_subsampling_max_mem_usage": 15000000000.0,
      "holdout_param": 0.1,
      "label": "Risk",
      "learning_type": "classification",
      "max_num_daub_ensembles": 2.0,
      "positive_label": "No Risk",
      "run_cognito_flag": true,
      "scorer_for_ranking": "accuracy"
   },
   "n_parallel_data_connections": 4.0,
   "one_vs_all_file": true
}
Optimization parameters: {
   "compute_pipeline_notebooks_flag": true,
   "cv_num_folds": 3.0,
   "daub_adaptive_subsampling_max_mem_usage": 15000000000.0,
   "holdout_param": 0.1,
   "label": "Risk",
   "learning_type": "classification",
   "max_num_daub_ensembles": 2.0,
   "positive_label": "No Risk",
   "run_cognito_flag": true,
   "scorer_for_ranking": "accuracy"
}


#### Find the AutoAI experiment runs matching the extracted pipeline

Extract the `project_id` where the training took place.

**Note:** If the training took place in the space please update accordingly.

In [16]:
training_project_id = deployed_pipeline_details['metadata']['tags'][0].split('.')[1]

List all training runs matching this pipeline name.

In [53]:
from ibm_watsonx_ai.experiment import AutoAI

experiment = AutoAI(credentials, project_id=training_project_id)
runs_df = experiment.runs(filter=deployed_pipeline_details['metadata']['name']).list()
runs_df.head()

,timestamp,run_id,state,auto_pipeline_optimizer name
0,2023-02-15T14:43:10.968Z,1736dbe2-e0d0-4eb5-ae88-34bb49934b9a,completed,Fairness - credit risk
1,2023-02-15T10:51:43.282Z,3d22ba3d-fda1-406e-b58d-f54e5b73e86b,completed,Fairness - credit risk
2,2023-02-15T10:45:47.522Z,8e60c682-3620-4564-b32b-53afd947db32,completed,Fairness - credit risk
3,2023-02-13T13:31:59.291Z,89fb81f6-1e07-431f-a276-4d97d3c34ac4,completed,Fairness - credit risk
4,2023-02-13T13:25:45.158Z,eaf3524e-ee07-4d12-8d2c-4e788f6cf4cc,completed,Fairness - credit risk


#### Extract AutoAI experiment `training_id`

Check if the `training_id` is available in model's details.

If not, we will use the latest run of AutoAI experiment matching the pipeline name.

**Hint:** It is also possible to extract the `run_id` parsing model's details (`location` property):
```
deployed_model_details['entity']['metrics'][0]['context']['intermediate_model']['location']['model'].split('wml_data/')[1].split('/data')[0]
```

In [54]:
try:
    run_id = deployed_model_details["entity"]["training_id"]
    print('AutoAI experiment training_id found in model details:', run_id)
except:
    run_id = runs_df['run_id'].values[0]
    print('AutoAI experiment training_id extracted from historical runs:', run_id)

AutoAI experiment training_id extracted from historical runs: 1736dbe2-e0d0-4eb5-ae88-34bb49934b9a


Get the training definition using `pipeline_id` linked with experiment run.

**NOTE:** We need to extract the `pipeline_id` from the same context as experiment run (`project`). The one linked with deployed model has been promoted to space and cannot be used for re-training purposes (different scope than training data reference).

In [55]:
pipeline_id = client.training.get_details(run_id)['entity']['pipeline']['id']
print('AutoAI experiment definition id:', pipeline_id)

AutoAI experiment definition id: 17edcc72-6b4f-4e83-b67a-9d71a57ce3eb


<a id="run"></a>
## 4. Experiment re-run 
Set the training `project_id` (where data asset resides) to retrain AutoAI models.


In [56]:
client.set.default_project(training_project_id)

metadata = {
    client.training.ConfigurationMetaNames.NAME: deployed_pipeline_details['metadata']['name'],
    client.training.ConfigurationMetaNames.DESCRIPTION: 'Re-train of ' + run_id,
    client.training.ConfigurationMetaNames.PIPELINE: {"id": pipeline_id},
    client.training.ConfigurationMetaNames.TRAINING_DATA_REFERENCES: deployed_model_details['entity']['training_data_references'],
    client.training.ConfigurationMetaNames.TRAINING_RESULTS_REFERENCE: {'location': {'path': '.'},
                                                                        'type': 'container'},
}

train_details = client.training.run(metadata, asynchronous=False)

Unsetting the space_id ...


##############################################

Running '638c4f4c-e6d5-4390-b8c7-5def49255ba4'

##############################################


pending........
running.....................
completed
Training of '638c4f4c-e6d5-4390-b8c7-5def49255ba4' finished successfully.


### Explore experiment's results
Connect to finished experiment and preview the results.

In [57]:
optimizer = experiment.runs.get_optimizer(run_id=train_details['metadata']['id'])
optimizer.summary()

,Enhancements,Estimator,training_roc_auc,holdout_average_precision,holdout_log_loss,training_accuracy_(optimized),holdout_roc_auc,training_balanced_accuracy,training_f1,holdout_precision,training_average_precision,training_log_loss,holdout_recall,training_precision,holdout_accuracy,holdout_balanced_accuracy,training_recall,holdout_f1
Pipeline Name,,,,,,,,,,,,,,,,,,
Pipeline_3,"HPO, FE",ExtraTreesClassifier,0.838711,0.496197,0.495841,0.764260,0.806569,0.740174,0.820847,0.797508,0.906944,0.467619,0.775758,0.828167,0.720322,0.693268,0.813682,0.786482
Pipeline_4,"HPO, FE, HPO",ExtraTreesClassifier,0.838711,0.496197,0.495841,0.764260,0.806569,0.740174,0.820847,0.797508,0.906944,0.467619,0.775758,0.828167,0.720322,0.693268,0.813682,0.786482
Pipeline_1,,ExtraTreesClassifier,0.841085,0.491631,0.487153,0.759339,0.817510,0.740409,0.814924,0.810726,0.910321,0.463011,0.778788,0.832392,0.732394,0.709753,0.798182,0.794436
Pipeline_2,HPO,ExtraTreesClassifier,0.841085,0.491631,0.487153,0.759339,0.817510,0.740409,0.814924,0.810726,0.910321,0.463011,0.778788,0.832392,0.732394,0.709753,0.798182,0.794436
Pipeline_8,"HPO, FE, HPO",SnapSVMClassifier,0.826571,0.498237,5.093310,0.750168,0.799347,0.741873,0.802920,0.804954,0.894194,NaN,0.787879,0.842682,0.732394,0.705317,0.767179,0.796325
Pipeline_7,"HPO, FE",SnapSVMClassifier,0.834588,0.498113,4.899148,0.749723,0.799038,0.743674,0.801723,0.830986,0.902161,NaN,0.715152,0.845704,0.714286,0.713863,0.762132,0.768730
Pipeline_6,HPO,SnapSVMClassifier,0.836787,0.496923,4.945203,0.746367,0.801579,0.743116,0.797672,0.830389,0.903943,NaN,0.712121,0.848040,0.712274,0.712348,0.753031,0.766721
Pipeline_5,,SnapSVMClassifier,0.824036,0.499548,5.108852,0.736301,0.797024,0.737670,0.786754,0.827465,0.894458,NaN,0.712121,0.848881,0.710262,0.709354,0.733490,0.765472


### Evaluate the best model locally

Load the model for test purposes.

**Hint:** The best model is returned automatically if no `pipeline_name` provided.

In [58]:
pipeline_name = 'Pipeline_4'
pipeline_model = optimizer.get_pipeline(pipeline_name=pipeline_name, astype='sklearn')
pipeline_model

Pipeline(steps=[('featureunion',
                 FeatureUnion(transformer_list=[('float32_transform_140373773717792',
                                                 Pipeline(steps=[('numpycolumnselector',
                                                                  NumpyColumnSelector(columns=[0,
                                                                                               1,
                                                                                               2,
                                                                                               3,
                                                                                               5,
                                                                                               6,
                                                                                               7,
                                                                                               8,
                                                                                               9,
                                                                                               10,
                                                                                               11,
                                                                                               12,
                                                                                               13,
                                                                                               14,
                                                                                               15,
                                                                                               16,
                                                                                               17,
                                                                                               18])),
                                                                 ('compressstrings',
                                                                  CompressStrings(compress_type='hash',
                                                                                  dtypes_list=['char_str',
                                                                                               'int_num',
                                                                                               'char_str',
                                                                                               'char_str',
                                                                                               'char_str',
                                                                                               'char_str',...
                 autoai_libs.cognito.transforms.transform_utils.TAM(tans_class = <class 'autoai_libs.cognito.transforms.transform_extras.IsolationForestAnomaly'>, name = 'isoforestanomaly', tgraph = None, apply_all = True, col_names = ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker', 'pca_0', 'pca_1', 'pca_3', 'pca_4', 'pca_5', 'pca_7', 'pca_8', 'pca_9', 'pca_10', 'pca_11', 'pca_12', 'pca_13', 'pca_14', 'pca_15', 'pca_17'], col_dtypes = [dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype(

This cell constructs the cell scorer based on the experiment metadata.

In [59]:
from sklearn.metrics import get_scorer

scorer = get_scorer(optimization_params['scorer_for_ranking'])

#### Read the train and holdout data.

**Hint:** You can also use external test dataset.

In [60]:
connection = optimizer.get_data_connections()[0]
train_X, test_X, train_y, test_y = connection.read(with_holdout_split=True)

#### Calculate the score

In [61]:
score = scorer(pipeline_model, test_X.values, test_y.values)
print(score)

0.7203219315895373


<a id="persistence"></a>

## 5. Store the model in repository

Provide `pipeline_name` and `training_id`.

In [62]:
model_metadata = {
    client.repository.ModelMetaNames.NAME: "{0} - {1} - {2}".format(deployed_pipeline_details['metadata']['name'],
                                                                    pipeline_name,
                                                                    pipeline_model.get_params()['steps'][-1][0])
}
published_model = client.repository.store_model(model=pipeline_name, meta_props=model_metadata, training_id=train_details['metadata']['id'])
updated_model_id = client.repository.get_model_id(published_model)
print('Re-trained model id', updated_model_id)


Re-trained model id ace63907-95e8-44d8-82c4-953e6d02339f


List stored models.

In [ ]:
client.repository.list_models()

<a id="redeploy"></a>
## 6. Redeploy and score new version of the model

In this section, you'll learn how to redeploy new version of the model by using the Watson Machine Learning Client.

**Hint:** As a best practice please consider using the test space before moving to production.

```
promote(asset_id: str, source_project_id: str, target_space_id: str, rev_id: str = None)
```

### Promote model to deployment space

In [64]:
promoted_model_id = client.spaces.promote(asset_id=updated_model_id, source_project_id=training_project_id, target_space_id=deployment_space_id)

Check current deployment details before update.

In [65]:
client.set.default_space(deployment_space_id)
print(json.dumps(client.deployments.get_details(deployment_id), indent=3))

Unsetting the project_id ...
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
{
   "entity": {
      "asset": {
         "id": "ce9c15b6-1480-4ec8-a60c-de7991463cea"
      },
      "custom": {},
      "deployed_asset_type": "model",
      "hybrid_pipeline_hardware_specs": [
         {
            "hardware_spec": {
               "id": "e7ed1d6c-2e89-42d7-aed5-863b972c1d2b",
               "name": "S",
               "num_nodes": 2
            },
            "node_runtime_id": "auto_ai.kb"
         }
      ],
      "name": "Fairness - credit risk - P4 Snap Random Forest Classifier",
      "online": {},
      "space_id": "efb29ab8-0862-4f0b-aeac-c523ee3d9258",
      "status": {
         "message": {
            "level": "warning",
            "text": "Successfully patched the asset."
         },
         "online_url": {
            "url": "https://us-south.ml.cloud.ibm.com/ml/v4/deployments/cd927dbd-b8e5-49f3-80f5-9965fb030348/predictions

### Update the deployment with new model
**Note:** The update is asynchronous.

In [66]:
metadata = {
    client.deployments.ConfigurationMetaNames.ASSET: {
        "id": promoted_model_id,
    }
}
    
updated_deployment = client.deployments.update(deployment_id, changes=metadata)

Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
Since ASSET is patched, deployment with new asset id/rev is being started. Monitor the status using deployments.get_details(deployment_uid) api


Wait for the deployment update: 

In [ ]:
import time

status = None
while status not in ['ready', 'failed']:
    time.sleep(2)
    deployment_details = client.deployments.get_details(deployment_id)
    status = deployment_details['entity']['status'].get('state')
    print('.', status, end=' ')
    
print("\nDeployment update finished with status: ", status)

#### Get updated deployment details

In [68]:
print(json.dumps(client.deployments.get_details(deployment_id), indent=2))

Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
{
  "entity": {
    "asset": {
      "id": "3b7f0c67-db45-4463-86c6-1dc4e9686787"
    },
    "custom": {},
    "deployed_asset_type": "model",
    "hybrid_pipeline_hardware_specs": [
      {
        "hardware_spec": {
          "id": "e7ed1d6c-2e89-42d7-aed5-863b972c1d2b",
          "name": "S",
          "num_nodes": 2
        },
        "node_runtime_id": "auto_ai.kb"
      }
    ],
    "name": "Fairness - credit risk - P4 Snap Random Forest Classifier",
    "online": {},
    "space_id": "efb29ab8-0862-4f0b-aeac-c523ee3d9258",
    "status": {
      "message": {
        "level": "warning",
        "text": "Successfully patched the asset."
      },
      "online_url": {
        "url": "https://us-south.ml.cloud.ibm.com/ml/v4/deployments/cd927dbd-b8e5-49f3-80f5-9965fb030348/predictions"
      },
      "serving_urls": [
        "https://us-south.ml.cloud.ibm.com/ml/v4/deployments/cd927dbd-b8

### Score updated model
Create sample payload and score the deployed model.

In [69]:
scoring_payload = {"input_data": [{"values": test_X[:3]}]}

Use client.deployments.score() method to run scoring.

In [70]:
predictions = client.deployments.score(deployment_id, scoring_payload)

In [71]:
print(json.dumps(predictions, indent=2))

{
  "predictions": [
    {
      "fields": [
        "prediction",
        "probability"
      ],
      "values": [
        [
          "No Risk",
          [
            0.5881121028804668,
            0.4118878971195331
          ]
        ],
        [
          "Risk",
          [
            0.029820769605171344,
            0.9701792303948288
          ]
        ],
        [
          "Risk",
          [
            0.29487495440414097,
            0.7051250455958591
          ]
        ]
      ]
    }
  ]
}


<a id="cleanup"></a>
## 7. Clean up

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- models
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 8. Summary and next steps

You successfully completed this notebook! You learned how to use scikit-learn machine learning as well as Watson Machine Learning for model creation and deployment. Check out our _[Online Documentation](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/welcome-main.html?context=wx)_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Lukasz Cmielowski**, PhD, is a Senior Technical Staff Member at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

Copyright © 2023-2024 IBM. This notebook and its source code are released under the terms of the MIT License.